In [1]:

#rest of your code.

In [2]:

from dotenv import load_dotenv
load_dotenv() #loads the env variables from the .env file

import os, sys
sys.path.append("..")
import emtmlibpy.emtmlibpy as emtm

In [3]:
key1 = os.environ["EMTMLIB_KEY1"]
key2 = os.environ["EMTMLIB_KEY2"]
success = emtm.emtm_set_licence_keys(key1, key2)
assert success, "Error setting EMTMlib licence keys"

In [4]:
print(emtm.emtm_version())

(4, 0)


In [5]:
em_files = ['/home/marrabld/projects/emtmlibpy/src/test_files/dpird/test_2/12099_C1H_23.11.23_0345.emobs',
            '/home/marrabld/projects/emtmlibpy/src/test_files/dpird/test_2/12099_C1H_23.11.23_0415.emobs',
            '/home/marrabld/projects/emtmlibpy/src/test_files/dpird/test_2/12099_C1H_23.11.23_0515.emobs',
            '/home/marrabld/projects/emtmlibpy/src/test_files/dpird/test_2/12099_C1H_23.11.23_0545.emobs']

In [6]:
for ii, em_file in enumerate(em_files):
    r = emtm.em_load_data(filename=em_file, em_file_id=ii)
    point_count, box_count = emtm.em_point_count(em_file_id=ii)
    print(f'number of points :: {point_count}')
    print(f'number of boxes :: {box_count}')

number of points :: 1326
number of boxes :: 1326
number of points :: 2232
number of boxes :: 2232
number of points :: 5925
number of boxes :: 5925
number of points :: 8383
number of boxes :: 8383


In [7]:
# this needs to be called first to generate the list
n_fgs = emtm.em_unique_fgs(em_file_id=0)
fgs = []

for ii in range(n_fgs):
    fgs.append(emtm.em_get_unique_fgs(n_index=ii, em_file_id=0))

print(fgs)

[('elasmobranch', 'na', 'gummywhiskery'), ('heterodontidae', 'heterodontus', 'portusjacksoni')]


In [8]:
sf = emtm.em_create(em_file_id=3)

In [9]:
data = emtm.em_get_point(em_file_id=0, n_index=0)
print(data)

In [10]:
r = emtm.em_add_point(em_file_id=3, data=data)


In [11]:
sf = emtm.em_create(em_file_id=4)

for ii, em_file in enumerate(em_files):
    r = emtm.em_load_data(filename=em_file, em_file_id=ii)
    point_count, box_count = emtm.em_point_count(em_file_id=ii)
    for jj in range(box_count):
        r = emtm.em_get_point(em_file_id=ii, n_index=jj)
        print(r.str_filename, r.n_frame, r.str_species)
        r = emtm.em_add_point(em_file_id=4, data=data)



b'12099_C1H_23.11.23_0345.mp4' 67 b'GummyWhiskery'
b'12099_C1H_23.11.23_0345.mp4' 69 b'GummyWhiskery'
b'12099_C1H_23.11.23_0345.mp4' 201 b'portusjacksoni'
b'12099_C1H_23.11.23_0345.mp4' 209 b'GummyWhiskery'
b'12099_C1H_23.11.23_0345.mp4' 212 b'portusjacksoni'
b'12099_C1H_23.11.23_0345.mp4' 213 b'portusjacksoni'
b'12099_C1H_23.11.23_0345.mp4' 221 b'portusjacksoni'
b'12099_C1H_23.11.23_0345.mp4' 224 b'portusjacksoni'
b'12099_C1H_23.11.23_0345.mp4' 225 b'portusjacksoni'
b'12099_C1H_23.11.23_0345.mp4' 228 b'GummyWhiskery'
b'12099_C1H_23.11.23_0345.mp4' 246 b'portusjacksoni'
b'12099_C1H_23.11.23_0345.mp4' 255 b'portusjacksoni'
b'12099_C1H_23.11.23_0345.mp4' 262 b'portusjacksoni'
b'12099_C1H_23.11.23_0345.mp4' 262 b'portusjacksoni'
b'12099_C1H_23.11.23_0345.mp4' 263 b'portusjacksoni'
b'12099_C1H_23.11.23_0345.mp4' 264 b'portusjacksoni'
b'12099_C1H_23.11.23_0345.mp4' 264 b'portusjacksoni'
b'12099_C1H_23.11.23_0345.mp4' 265 b'portusjacksoni'
b'12099_C1H_23.11.23_0345.mp4' 266 b'portusjacksoni'

In [12]:
# emtm.em_write_data(em_file_id=4, filename='concat_emobs.EMObs')


In [17]:
def concat_emobs(files: list, output_file: str):
    """
    Given a list of emobs files, concatenate them and save as the new output file. 

    Requires write access in the licence file. 
    """
    DEBUG = True
    num_files = len(files)
    out_file_id = num_files + 1

    sf = emtm.em_create(em_file_id=out_file_id)

    for in_file_id, em_file in enumerate(em_files):
        print(f'getting points for {em_file}')
        r = emtm.em_load_data(filename=em_file, em_file_id=in_file_id)
        point_count, box_count = emtm.em_point_count(em_file_id=in_file_id)
        for jj in range(box_count):
            data = emtm.em_get_point(em_file_id=in_file_id, n_index=jj)
            if DEBUG:
                print(f'file : {data.str_filename} frame : {data.n_frame} species : {data.str_species}')
            r = emtm.em_add_point(em_file_id=out_file_id, data=data)

    print(f'writing to file {output_file}')
    return emtm.em_write_data(em_file_id=out_file_id, filename=output_file)
    



In [18]:
r = concat_emobs(em_files, output_file="v3_test_out.EMObs")
print(r)

getting points for /home/marrabld/projects/emtmlibpy/src/test_files/dpird/test_2/12099_C1H_23.11.23_0345.emobs
file : b'12099_C1H_23.11.23_0345.mp4' frame : 67 species : b'GummyWhiskery'
file : b'12099_C1H_23.11.23_0345.mp4' frame : 69 species : b'GummyWhiskery'
file : b'12099_C1H_23.11.23_0345.mp4' frame : 201 species : b'portusjacksoni'
file : b'12099_C1H_23.11.23_0345.mp4' frame : 209 species : b'GummyWhiskery'
file : b'12099_C1H_23.11.23_0345.mp4' frame : 212 species : b'portusjacksoni'
file : b'12099_C1H_23.11.23_0345.mp4' frame : 213 species : b'portusjacksoni'
file : b'12099_C1H_23.11.23_0345.mp4' frame : 221 species : b'portusjacksoni'
file : b'12099_C1H_23.11.23_0345.mp4' frame : 224 species : b'portusjacksoni'
file : b'12099_C1H_23.11.23_0345.mp4' frame : 225 species : b'portusjacksoni'
file : b'12099_C1H_23.11.23_0345.mp4' frame : 228 species : b'GummyWhiskery'
file : b'12099_C1H_23.11.23_0345.mp4' frame : 246 species : b'portusjacksoni'
file : b'12099_C1H_23.11.23_0345.mp4'